<a href="https://colab.research.google.com/github/VEERA-10/Library_management_system/blob/main/LIBRARYMANAGEMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from datetime import date, timedelta

def create_database():
    connection = sqlite3.connect("library.db")
    cursor = connection.cursor()

    # Create tables if they don't exist
    cursor.execute(open("schema.sql", "r").read())

    connection.commit()
    connection.close()

def add_book(title, author, genre, quantity_available):
    connection = sqlite3.connect("library.db")
    cursor = connection.cursor()

    cursor.execute("INSERT INTO books (title, author, genre, quantity_available) VALUES (?, ?, ?, ?)",
                   (title, author, genre, quantity_available))

    connection.commit()
    connection.close()

def add_patron(name, email):
    connection = sqlite3.connect("library.db")
    cursor = connection.cursor()

    cursor.execute("INSERT INTO patrons (name, email) VALUES (?, ?)", (name, email))

    connection.commit()
    connection.close()

def borrow_book(book_id, patron_id):
    connection = sqlite3.connect("library.db")
    cursor = connection.cursor()

    borrowing_date = date.today()
    due_date = borrowing_date + timedelta(days=14)  # Due date is 14 days from borrowing date
    cursor.execute("INSERT INTO borrowings (book_id, patron_id, borrowing_date, due_date) VALUES (?, ?, ?, ?)",
                   (book_id, patron_id, borrowing_date, due_date))

    # Update quantity_available in books table
    cursor.execute("UPDATE books SET quantity_available = quantity_available - 1 WHERE book_id = ?", (book_id,))

    connection.commit()
    connection.close()

def return_book(borrowing_id):
    connection = sqlite3.connect("library.db")
    cursor = connection.cursor()

    return_date = date.today()
    cursor.execute("UPDATE borrowings SET return_date = ? WHERE borrowing_id = ?", (return_date, borrowing_id))

    # Calculate overdue charges, if any
    cursor.execute("SELECT due_date FROM borrowings WHERE borrowing_id = ?", (borrowing_id,))
    due_date = cursor.fetchone()[0]
    days_late = max((return_date - due_date).days, 0)
    overdue_charge = 0.25 * days_late
    cursor.execute("UPDATE borrowings SET overdue_charge = ? WHERE borrowing_id = ?", (overdue_charge, borrowing_id))

    # Update quantity_available in books table
    cursor.execute("""
        UPDATE books
        SET quantity_available = quantity_available + 1
        WHERE book_id = (SELECT book_id FROM borrowings WHERE borrowing_id = ?)
    """, (borrowing_id,))

    connection.commit()
    connection.close()

if __name__ == "__main__":
    create_database()

    # Example usage
    add_book("Book A", "Author X", "Fiction", 5)
    add_patron("John Doe", "john@example.com")
    borrow_book(1, 1)
    return_book(1)
